# Kaggle Competition- Blue book for bulldozers

### Date: 20/04/2019
### Author: Aakash Nand
### Goal: Goal of this notebook is to get started with RandomForest Algorithm for regression using fastai course 
( http://course18.fast.ai )


## Table of Contents
1. [What is Blue book for bullldozers?](#intro)
2. [Evaluation of predictions](#eval) 
3. [Getting started](#start)
4. [Initial setup](#setup)<br>
    4.1 [Downloading the data](#download)<br>
    4.2 [Importing the libraries](#imports)<br>
    4.3 [Loading the data](#loading-data)<br>
    4.4 [Peek into the data](#peek)
5. [Preprocessing](#preprocessing)<br>
    5.1 [Extracting information from dates](#date-features)<br>
    5.2 [Changing to Categorical Variables](#categorical)<br>
    5.3 [Setting correct order for categorical variable](#correct-order)<br>
    5.4 [Handling Missing Values and Categorical variables codes](#missing-values)<br>
    5.5 [Saving Dataframe to harddisk for faster access](#saving-df)

<a id="intro"></a>
## What is Blue book for bulldozers?

The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations. [Fast Iron](https://www.fastiron.com/Default.aspx) is creating a "blue book for bull dozers," for customers to value what their heavy equipment fleet is worth at auction.

<b>Note: Currently this competition is closed for submissions</b>

<a id="eval"></a>
## Evaluation of predictions:
The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

Sample submission files can be downloaded from the data page. Submission files should be formatted as follows:

Have a header: "SalesID,SalePrice"
Contain two columns
SalesID: SalesID for the validation set in sorted order
SalePrice: Your predicted price of the sale
```
SalesID,SalePrice
1222837,36205
3044012,74570
1222841,31910.50
... 
```

<a id="start"></a>
## Lets get started !!!

<a id="setup"></a>
## Initial Setup

<a id="download"></a>
### Downloading the data
We can use kaggle api for downloading the dataset. Please follow kaggle's instruction on how to setup kaggle api

`kaggle competitions download -c bluebook-for-bulldozers`

<a id="imports"></a>
### Importing the libraries

In [2]:
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import metrics
import re
from pandas.api.types import *
import os

<a id="loading-data"></a>
### Loading the data
We will use the `df_raw` as raw dataframe throughout the notebook

In [132]:
PATH="./data/"
df_raw=pd.read_csv(f'{PATH}Train.csv',parse_dates=["saledate"],low_memory=False)

<a id="peek"></a>
### Peek into the data
Since there are many columns lets use transpose view

In [81]:
df_raw.head().transpose(copy=True)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


<a id="preprocessing"></a>
### Preprocessing

It is important to note that this kaggle competition evaluates the performance of model using RMSLE (root mean squared log error). Hence we will convert our training data set dependent variable into log scale

In [133]:
df_raw.SalePrice=np.log(df_raw.SalePrice)

<a id='date-features'></a>
### Extracting information from dates
A date can reveal lot of important information such quarter, day of week etc. We will write a function which will retrieve this information and add it to dataframe

In [134]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, 
                                     infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 
            'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 
            'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    #df.columns()
    if drop: df.drop(fldname, axis=1, inplace=True)

add_datepart(df_raw,'saledate')

<a id ='categorical'></a>
### Changing to categorical variables

Machine learning models usually need lot of data preprocessing. This includes changing data types of columns of dataframe to make it compatible with standard models. Hence, in our case we need to change any `string` data type column to categorical data type.

In [135]:
for colnames, series in df_raw.items():
    if is_string_dtype(series):
        df_raw[colnames]=series.astype('category').cat.as_ordered()
df_raw.UsageBand.dtype

CategoricalDtype(categories=['High', 'Low', 'Medium'], ordered=True)

<a id='correct-order'></a>
### Setting correct order for categorical variable

As we can see from above output Column `UsageBand` has values such as `High`,`Medium` and `Low` which are currently soreted in alphabetical order which does not harm our model but let's change the order to symentically correct way that is `High`>`Medium`>`Low`.

In [136]:
df_raw.UsageBand.cat.set_categories(['High','Medium','Low'],ordered=True,inplace=True)
df_raw.UsageBand.dtype

CategoricalDtype(categories=['High', 'Medium', 'Low'], ordered=True)

<a id='missing-values'></a>
### Handling Missing Values and Categorical variables codes

We are not done yet. There are many null values in our data and categorical variables still represent categories hence we will use two strategies:
1. For Numerical column, we will create a new column with `this_column_name`+`_na` concatinated to it which will contain boolean values for presence of null value in `this_column`.<br>For ex. if the numerical column is `SaleId` with total 5 values and with `null` values at location 1,2 and 4 then we will add new column `UsageBand_na` with values ```SaleId_na=[True,True,False,True,False]```
2. For categorical columns, pandas add `-1` for missing values hence we will just add +1 to all values and copy those categorical codes to actual column which will make the original column numeric.<br> For ex. if the categorical column `UsageBand` has total 5 values with missing value at location 2 and 4 then original `UsageBand` will look like this ```UsageBand=[Low,-1,High,-1,Medium] ``` hence we will change the `UsageBand` to ```UsageBand=[2,0,4,0,3] where 2=Low,4=High, 3=Medium, 0=Missing ```
3. We will also remove the dependent variable from training dataset that is `SalePrice`

The following two functions handles missing values and categorical variable codes

In [137]:
def process_dataframe(df,y_field):
    y=df[y_field].values
    df.drop(y_field,axis=1,inplace=True)
    for label,content in df.items():
        fix_missing_values(df,label,content)
    result=[df,y]
    return result

def fix_missing_values(df,label,content):
    if is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df[label+'_na']=pd.isnull(content)
        df[label]=content.fillna(content.median())
    if is_categorical_dtype(content):
        df[label]=content.cat.codes+1
        

In [138]:
df, y = process_dataframe(df_raw, 'SalePrice')

<a id='saving-df'></a>
### Saving Dataframe to harddisk for faster access

At this point we are done with preprocessing of dataframe. Now is the correct time to save it storage file. For future access we can retrieve dataframe from this stage.

In [141]:
os.makedirs('tmp',exist_ok=True)
df.to_feather('tmp/processed_df')

### Reading back the dataframe from storage

In [3]:
df=pd.read_feather('tmp/processed_df')
df.dtypes

SalesID                          int64
MachineID                        int64
ModelID                          int64
datasource                       int64
auctioneerID                   float64
YearMade                         int64
MachineHoursCurrentMeter       float64
UsageBand                         int8
fiModelDesc                      int16
fiBaseModel                      int16
fiSecondaryDesc                  int16
fiModelSeries                     int8
fiModelDescriptor                int16
ProductSize                       int8
fiProductClassDesc                int8
state                             int8
ProductGroup                      int8
ProductGroupDesc                  int8
Drive_System                      int8
Enclosure                         int8
Forks                             int8
Pad_Type                          int8
Ride_Control                      int8
Stick                             int8
Transmission                      int8
Turbocharged             